<a href="https://colab.research.google.com/github/ccarpenterg/LearningPyTorch1.x/blob/master/04_cifar_10_challenging_convnets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### CIFAR-10: A More Challenging Dataset for CNNs

In [0]:
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

import statistics

In [0]:
class BasicCNN(nn.Module):
    
    def __init__(self, num_channels, num_classes):
        super(BasicCNN, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, 32, 3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(64, 64, 3, stride=1, padding=0)
        self.pool1 = nn.MaxPool2d(2)
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(4*4*64, 64, bias=True)
        self.fc2 = nn.Linear(64, 10)
        
    def forward(self, X):
        x = F.relu(self.conv1(X))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = x.reshape(-1, 4*4*64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

In [3]:
cuda = torch.device('cuda')

model = BasicCNN(3, 10)
model.to(cuda)

BasicCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

In [4]:
dataset_transform = transforms.Compose([
    transforms.ToTensor()
])

train_set = CIFAR10('./cifar10', train=True, download=True, transform=dataset_transform)
valid_set = CIFAR10('./cifar10', train=False, download=True, transform=dataset_transform)

print(train_set.data.shape)
print(valid_set.data.shape)

  0%|          | 0/170498071 [00:00<?, ?it/s]

170500096it [00:01, 86022492.04it/s]                               


Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [0]:
train_loader = DataLoader(train_set, batch_size=128, num_workers=0, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=512, num_workers=0, shuffle=False)

In [6]:
x = torch.randn(128, 3, 32, 32, device=cuda)
output = model(x)
print(output.shape)

torch.Size([128, 10])


In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [0]:
def train(model, loss_fn, optimizer):
    model.train()
    
    train_batch_losses = []
    
    for batch, labels in train_loader:
        batch = batch.to(cuda)
        labels = labels.to(cuda)
        
        optimizer.zero_grad()
        y_pred = model(batch)
        loss = loss_fn(y_pred, labels)
        loss.backward()
        optimizer.step()
        
        train_batch_losses.append(float(loss))
        
        mean_loss = statistics.mean(train_batch_losses)
        
    return mean_loss

In [0]:
def validate(model, loss_fn, optimizer):
    model.eval()
    
    predictions = []
    
    with torch.no_grad():
        validation_batch_losses = []
        
        for batch, labels in valid_loader:
            batch = batch.to(cuda)
            labels = labels.to(cuda)
            
            labels_pred = model(batch)
            loss = loss_fn(labels_pred, labels)
            
            validation_batch_losses.append(float(loss))
            
            mean_loss = statistics.mean(validation_batch_losses)
            
    return mean_loss

In [0]:
def accuracy(model, loader):
    correct = 0
    total = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch, labels in loader:
            batch = batch.to(cuda)
            labels = labels.to(cuda)
        
            labels_pred = model(batch)
        
            _, predicted = torch.max(labels_pred.data, 1)
        
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
    return (100 * correct / total)

In [16]:
loss_fn = nn.CrossEntropyLoss()

train_losses = []
valid_losses = []

for epoch in range(1, 1+15):
    
    print('Epoch: ', epoch)
    
    train_loss = train(model, loss_fn, optimizer)
    train_losses.append(train_loss)
    
    print('Training loss:', train_loss)
    print('Training accuracy: {}%'.format(accuracy(model, train_loader)))
    
    valid_loss = validate(model, loss_fn, optimizer)
    valid_losses.append(valid_loss)
    
    print('Validation loss:', valid_loss)
    print('Validation accuracy: {}%'.format(accuracy(model, valid_loader)))

Epoch:  1
Training loss: 1.1408028633088407
Training accuracy: 65.464%
Validation loss: 1.2514345467090606
Validation accuracy: 58.61%
Epoch:  2
Training loss: 1.1092442223787917
Training accuracy: 63.538%
Validation loss: 1.3400908768177033
Validation accuracy: 55.2%
Epoch:  3
Training loss: 1.119702333227143
Training accuracy: 64.53%
Validation loss: 1.345867383480072
Validation accuracy: 55.97%
Epoch:  4
Training loss: 1.1152159844518013
Training accuracy: 61.222%
Validation loss: 1.4285688281059266
Validation accuracy: 53.75%
Epoch:  5
Training loss: 1.0982776875691036
Training accuracy: 66.952%
Validation loss: 1.3548159956932069
Validation accuracy: 57.51%
Epoch:  6
Training loss: 1.1000229629409282
Training accuracy: 61.238%
Validation loss: 1.4709051728248597
Validation accuracy: 53.16%
Epoch:  7
Training loss: 1.1492690730582722
Training accuracy: 62.446%
Validation loss: 1.3778671860694884
Validation accuracy: 54.4%
Epoch:  8
Training loss: 1.1143412532099068
Training accurac